In [2]:
import pandas as pd
from ast import literal_eval
from sympy.combinatorics import Permutation
from dataclasses import dataclass
from typing import Dict, List


In [3]:
puzzle_info_path = "./data/puzzle_info.csv"
puzzles_path = "./data/puzzles.csv"
sample_submission_path = "./data/sample_submission.csv"
solutions_path = "./solutions"

In [4]:
puzzle_info = pd.read_csv("./data/puzzle_info.csv", index_col='puzzle_type')
all_allowed_moves = {}
for idx, row in puzzle_info.iterrows():
    moves = literal_eval(puzzle_info.loc[idx, 'allowed_moves'])
    moves = {k: Permutation(v) for k, v in moves.items()}
    all_allowed_moves[idx] = moves

In [5]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
puzzles = pd.read_csv("./data/puzzles.csv")

### Apply a solution and check

**Module**

In [6]:
@dataclass
class Puzzle:
    """A permutation puzzle."""

    puzzle_id: str
    solution_state: List[str]
    initial_state: List[str]
    num_wildcards: int

class ParticipantVisibleError(Exception):
    pass


In [7]:

def score_puzzle(puzzle_id, puzzle, sub_solution):
    """Score the solution to a permutation puzzle."""
    # Apply submitted sequence of moves to the initial state, from left to right
    moves = sub_solution.split('.')
    state = puzzle.initial_state
    for m in moves:
        power = 1
        if m[0] == "-":
            m = m[1:]
            power = -1
        try:
            p = all_allowed_moves[puzzle_id][m]
        except KeyError:
            raise ParticipantVisibleError(f"{m} is not an allowed move for {puzzle_id}.")
        state = (p ** power)(state)

    # Check that submitted moves solve puzzle
    num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
    if num_wrong_facelets > puzzle.num_wildcards:
        raise ParticipantVisibleError(f"Submitted moves do not solve {puzzle_id}.")

    # The score for this instance is the total number of moves needed to solve the puzzle
    return len(moves)

**Code**

In [8]:
submission = pd.read_csv(sample_submission_path)
puzzles = pd.read_csv(puzzles_path)
total_num_moves = 0
for puzzle_row, solution_row in zip(puzzles.itertuples(), submission.itertuples()):
    puzzle_id = getattr(puzzle_row, "id")
    assert puzzle_id == getattr(solution_row, "id")
    puzzle_type = getattr(puzzle_row, "puzzle_type")
    puzzle = Puzzle(
        puzzle_id=puzzle_type,
        solution_state=puzzle_row.solution_state.split(';'),
        initial_state=puzzle_row.initial_state.split(';'),
        num_wildcards=puzzle_row.num_wildcards,
    )

        # Score submission row
    num_moves = score_puzzle(puzzle_type, puzzle, getattr(solution_row, "moves"))
    total_num_moves += num_moves

KeyboardInterrupt: 

In [ ]:
total_num_moves

## DFS

In [9]:
nodes = [(puzzle.initial_state, [])]

In [10]:
print(puzzles.loc[0].solution_state.split(';'))
print(puzzles.loc[0].initial_state.split(';'))

['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D', 'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C']


In [61]:
# puzzle = Puzzle(
#         puzzle_id="cube_2/2/2",
#         solution_state=['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F'],
#         initial_state=['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D', 'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C'],
#         num_wildcards=0
#     )
puzzle = Puzzle(
        puzzle_id="cube_2/2/2",
        solution_state="A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F".split(";"),
        initial_state="D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C".split(";"),
        num_wildcards=0
    )
num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, puzzle.initial_state))
nodes = [(puzzle.initial_state, tuple(), num_wrong_facelets)]
iterations = 0
best_sol_for_state = dict()
best_upper_bound = 63
def get_deviation(e):
    return -e[-1]
while len(nodes):
    solution_found = False
    # nodes.sort(key = get_deviation)
    cur_state, moves_seq, num_wrong_facelets = nodes.pop()
    num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, cur_state))
    if num_wrong_facelets <= puzzle.num_wildcards:
        print("Solution Found.")
        print(moves_seq)
        solution_found = True
    moves = all_allowed_moves[puzzle.puzzle_id]
    for m, perm in moves.items():
        if len(moves_seq) > 0 and (m == moves_seq[-1] or "-"+m == moves_seq[-1]):
            continue
        state = (perm)(cur_state)
        new_moves_seq = moves_seq + (m,)
        num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
        if num_wrong_facelets <= puzzle.num_wildcards:
            print("Solution Found.")
            print(new_moves_seq)
            solution_found = True
        flag = True
        if tuple(state) in best_sol_for_state:
            if best_sol_for_state[tuple(state)] < len(new_moves_seq) or len(new_moves_seq) > best_upper_bound:
                flag = False
        if flag:
            if len(new_moves_seq) <= best_upper_bound:
                nodes.append((state, new_moves_seq, num_wrong_facelets))
                best_sol_for_state[tuple(state)] = len(new_moves_seq)
                iterations += 1
        state = (perm ** -1)(cur_state)
        new_moves_seq = moves_seq + ("-"+m,)
        num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
        if num_wrong_facelets <= puzzle.num_wildcards:
            print("Solution Found.")
            print(new_moves_seq)
            solution_found = True
        flag = True

        if tuple(state) in best_sol_for_state:
            if best_sol_for_state[tuple(state)] < len(new_moves_seq) or len(new_moves_seq) > best_upper_bound:
                flag = False
        if flag:
            if len(new_moves_seq) <= best_upper_bound:
                nodes.append((state, new_moves_seq, num_wrong_facelets))
                best_sol_for_state[tuple(state)] = len(new_moves_seq)
                iterations += 1
    # if(iterations > 10000000):
    #     print(f"Limit exceeded: {iterations}")
    if(solution_found):
        print(f"Iterations: {iterations}")
        break

KeyboardInterrupt: 

In [64]:
len(nodes)

484

In [56]:
len(nodes)

543

## BFS

In [13]:
# puzzle = Puzzle(
#         puzzle_id="cube_2/2/2",
#         solution_state=['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F'],
#         initial_state=['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D', 'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C'],
#         num_wildcards=0
#     )
puzzle = Puzzle(
        puzzle_id="cube_2/2/2",
        solution_state="A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F".split(";"),
        initial_state="D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C".split(";"),
        num_wildcards=0
    )
num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, puzzle.initial_state))
nodes = [(puzzle.initial_state, tuple(), num_wrong_facelets)]
iterations = 0
best_sol_for_state = dict()
best_upper_bound = 63
def get_deviation(e):
    return -e[-1]
while len(nodes):
    solution_found = False
    # nodes.sort(key = get_deviation)
    cur_state, moves_seq, num_wrong_facelets = nodes.pop(0)
    num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, cur_state))
    if num_wrong_facelets <= puzzle.num_wildcards:
        print("Solution Found.")
        print(moves_seq)
        solution_found = True
    moves = all_allowed_moves[puzzle.puzzle_id]
    for m, perm in moves.items():
        if len(moves_seq) > 0 and (m == moves_seq[-1] or "-"+m == moves_seq[-1]):
            continue
        state = (perm)(cur_state)
        new_moves_seq = moves_seq + (m,)
        num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
        if num_wrong_facelets <= puzzle.num_wildcards:
            print("Solution Found.")
            print(new_moves_seq)
            solution_found = True
        flag = True
        if tuple(state) in best_sol_for_state:
            if best_sol_for_state[tuple(state)] < len(new_moves_seq) or len(new_moves_seq) > best_upper_bound:
                flag = False
        if flag:
            if len(new_moves_seq) <= best_upper_bound:
                nodes.append((state, new_moves_seq, num_wrong_facelets))
                best_sol_for_state[tuple(state)] = len(new_moves_seq)
                iterations += 1
        state = (perm ** -1)(cur_state)
        new_moves_seq = moves_seq + ("-"+m,)
        num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
        if num_wrong_facelets <= puzzle.num_wildcards:
            print("Solution Found.")
            print(new_moves_seq)
            solution_found = True
        flag = True

        if tuple(state) in best_sol_for_state:
            if best_sol_for_state[tuple(state)] < len(new_moves_seq) or len(new_moves_seq) > best_upper_bound:
                flag = False
        if flag:
            if len(new_moves_seq) <= best_upper_bound:
                nodes.append((state, new_moves_seq, num_wrong_facelets))
                best_sol_for_state[tuple(state)] = len(new_moves_seq)
                iterations += 1
    # if(iterations > 10000000):
    #     print(f"Limit exceeded: {iterations}")
    if(solution_found):
        print(f"Iterations: {iterations}")
        break

KeyboardInterrupt: 

In [22]:
x = "A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F".split(";")
y = (all_allowed_moves['cube_2/2/2']['f0'])(x)
print(x)
print(y)
print(all_allowed_moves['cube_2/2/2']['f0'])

['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
['A', 'A', 'E', 'E', 'B', 'B', 'B', 'B', 'A', 'C', 'A', 'C', 'D', 'D', 'D', 'D', 'E', 'F', 'E', 'F', 'C', 'C', 'F', 'F']
(23)(2 19 21 8)(3 17 20 10)(4 6 7 5)
